# How and Why BatchNorm Works - Part 2


In Part 1, we showed how using batch normalization can help train neural networks by reducing internal covariate shift and smoothing gradient updates. We also touch upon the idea that it makes the loss surface smoother and more convex (residual connections have a similar effect).

In their 2018 paper, Santukar et al. argue that internal covariate shift isn't substantially reduced at all. In fact, when additional noise is added to the activations to force covariate shift, batch normalization continues to improve training performance. This is despite no significant reduction in covariate shift. They argue that the major reason for improved performance is the smoother loss surface.

However, we saw in our own experiments using a shallow neural network that the reduction in covariate shift is substantial. Let's get to the bottom of this.

In this experiment, we will start by reproducing the results from Santukar et al. Then we will try and understand why their results and our previous experiment differ.

## Let's start as usual by setting up the dataset
We will use the CIFAR-10 dataset for this experiment.
The CIFAR-10 dataset is a widely used benchmark in machine learning for evaluating image classification algorithms. It consists of 60,000 color images, each sized 32x32 pixels, divided into 10 distinct classes such as airplanes, automobiles, birds, cats, and ships. The dataset is split into 50,000 training images and 10,000 test images, with an equal number of examples per class. Created by the Canadian Institute for Advanced Research, CIFAR-10 is challenging due to its low resolution and high intra-class variability. It serves as a foundational dataset for developing and comparing deep learning models, especially convolutional neural networks (CNNs).

In [1]:
def is_colab():
    try:
        import google.colab
        return True
    except ImportError:
        return False

if is_colab():
    !git clone https://github.com/novastar53/deepkit
    !cd deepkit && uv build . --wheel && pip install ./dist/*


fatal: destination path 'deepkit' already exists and is not an empty directory.
running egg_info
writing src/deepkit.egg-info/PKG-INFO
writing dependency_links to src/deepkit.egg-info/dependency_links.txt
writing requirements to src/deepkit.egg-info/requires.txt
writing top-level names to src/deepkit.egg-info/top_level.txt
reading manifest file 'src/deepkit.egg-info/SOURCES.txt'
adding license file 'LICENSE'
writing manifest file 'src/deepkit.egg-info/SOURCES.txt'
running bdist_wheel
running build
running build_py
copying src/deepkit/datasets.py -> build/lib/deepkit
copying src/deepkit/__init__.py -> build/lib/deepkit
running egg_info
writing src/deepkit.egg-info/PKG-INFO
writing dependency_links to src/deepkit.egg-info/dependency_links.txt
writing requirements to src/deepkit.egg-info/requires.txt
writing top-level names to src/deepkit.egg-info/top_level.txt
reading manifest file 'src/deepkit.egg-info/SOURCES.txt'
adding license file 'LICENSE'
writing manifest file 'src/deepkit.egg-inf

In [2]:
from deepkit import load_CIFAR10
train_loader, test_loader = load_CIFAR10()

/usr/local/lib/python3.11/dist-packages/jaxlib/plugin_support.py:71: RuntimeWarning: JAX plugin jax_cuda12_plugin version 0.5.1 is installed, but it is not compatible with the installed jaxlib version 0.5.3, so it will not be used.
  warnings.warn(


## Next, let's set up the model
We'll be using the standard VGG-Net architecture as per the paper.






In [3]:
import jax
import jax.numpy as jnp
import flax.nnx as nnx

class VGGBlock(nnx.Module):
    def __init__(self, in_features: int, out_features: int, rngs: nnx.Rngs):
        self.conv = nnx.Conv(in_features=in_features,
                             out_features=out_features,
                             kernel_size=(3, 3),
                             padding='SAME',
                             rngs=rngs)
        self.bn = nnx.BatchNorm(num_features=out_features, rngs=rngs)

    def __call__(self, x, training: bool):
        x = self.conv(x)
        x = self.bn(x)
        x = nnx.relu(x)
        return x

class VGGNet(nnx.Module):
    def __init__(self, rngs: nnx.Rngs):
        self.convs = [
            VGGBlock(in_features=3, out_features=64,  rngs=rngs),
            VGGBlock(in_features=64, out_features=128, rngs=rngs),
            VGGBlock(in_features=128, out_features=256, rngs=rngs),
            VGGBlock(in_features=256, out_features=256, rngs=rngs),
            VGGBlock(in_features=256, out_features=512, rngs=rngs),
            VGGBlock(in_features=512, out_features=512, rngs=rngs),
            VGGBlock(in_features=512, out_features=512, rngs=rngs),
            VGGBlock(in_features=512, out_features=512, rngs=rngs),
        ]

        self.fc1 = nnx.Linear(in_features=512, out_features=512, rngs=rngs)
        self.fc2 = nnx.Linear(in_features=512, out_features=512, rngs=rngs)
        self.out = nnx.Linear(in_features=512, out_features=10,  rngs=rngs)

    def __call__(self, x, training: bool = True):
        max_pool_after = [0, 1, 3, 5, 7]
        for layer in range(len(self.convs)):
            x = self.convs[layer](x, training)
            if layer in max_pool_after:
                x = nnx.max_pool(x, window_shape=(2, 2), strides=(2, 2))

        x = x.squeeze()
        x = nnx.relu(self.fc1(x))
        x = nnx.relu(self.fc2(x))
        x = self.out(x)
        return x

In [4]:
rng_key = jax.random.key(1337)
rngs = nnx.Rngs(rng_key)
candidate = VGGNet(rngs=rngs)
graphdef, state = nnx.split(candidate)
param_counts = sum(jax.tree_util.tree_leaves(jax.tree_util.tree_map(lambda x: x.size, state)))
print(f"Initialized model with {param_counts:,} parameters.")
nnx.display(state)

Initialized model with 9,761,930 parameters.


In [5]:
rng_key = jax.random.key(1337)
rngs = nnx.Rngs(rng_key)
baseline = VGGNet(rngs=rngs)

class Dummy(nnx.Module):
    def __call__(self, x):
        return x


for vgg in baseline.convs:
  vgg.bn = Dummy()
graphdef, state = nnx.split(baseline)
param_counts = sum(jax.tree_util.tree_leaves(jax.tree_util.tree_map(lambda x: x.size, state)))
print(f"Initialized model with {param_counts:,} parameters.")
nnx.display(state)

Initialized model with 9,750,922 parameters.


In [6]:
import optax

baseline_optimizer = nnx.Optimizer(baseline, optax.sgd(learning_rate=0.01, momentum=0.9))
candidate_optimizer = nnx.Optimizer(candidate, optax.sgd(learning_rate=0.01, momentum=0.9))

In [7]:
from functools import partial


def loss_fn(model, batch, targets):
    logits = model(batch)
    loss = optax.softmax_cross_entropy_with_integer_labels(logits, targets).mean()
    return loss

@nnx.jit
def step_fn(model: nnx.Module, optimizer: nnx.Optimizer, batch: jax.Array, labels: jax.Array):
    loss, grads = nnx.value_and_grad(loss_fn)(model, batch, labels)
    optimizer.update(grads)
    return loss, grads


@nnx.jit
def accuracy(model: nnx.Module, batch: jax.Array, labels: jax.Array):
    logits = model(batch)
    #probs = nnx.softmax(logits, axis=-1)
    preds = jnp.argmax(logits, axis=-1)
    sum = jnp.sum(preds == labels)
    acc = sum/logits.shape[0]
    return acc


def test_accuracy(model: nnx.Module, testloader):
    acc, n = 0, 0
    for batch, labels in testloader:
        batch = jnp.array(batch)
        labels = jnp.array(labels)
        acc += accuracy(model, batch, labels)
        n += 1
    return acc/n



In [8]:
num_epochs = 10
num_steps = num_epochs*len(train_loader)

In [ ]:
i = 0
baseline.train()
candidate.train()
for epoch in range(num_epochs):
    for batch, labels in train_loader:

        batch = jnp.array(batch)
        labels = jnp.array(labels)
        baseline.train()
        candidate.train()
        baseline_loss, _ = step_fn(baseline, baseline_optimizer, batch, labels)
        candidate_loss, _ = step_fn(candidate, candidate_optimizer, batch, labels)
        baseline.eval()
        candidate.eval()
        baseline_acc = accuracy(baseline, batch, labels)
        candidate_acc = accuracy(candidate, batch, labels)
        if i % 20 == 0:
          print(f"iter: {i} | baseline loss: {baseline_loss:0.4f} | candidate loss: {candidate_loss:0.4f} | baseline train acc: {baseline_acc:0.2f} | candidate train acc: {candidate_acc:0.2f}")
        if i % 200 == 0:
          baseline_test_acc = test_accuracy(baseline, test_loader)
          candidate_test_acc = test_accuracy(candidate, test_loader)
          print(f"iter: {i} | baseline test acc: {baseline_test_acc: 0.2f} | candidate test acc: {candidate_test_acc: 0.2f}")
        i += 1

iter: 0 | baseline loss: 2.2947 | candidate loss: 2.3680 | baseline train acc: 0.09 | candidate train acc: 0.08
iter: 0 | baseline test acc:  0.07 | candidate test acc:  0.07
iter: 20 | baseline loss: 2.2920 | candidate loss: 1.8441 | baseline train acc: 0.11 | candidate train acc: 0.22
iter: 40 | baseline loss: 2.2094 | candidate loss: 1.7026 | baseline train acc: 0.18 | candidate train acc: 0.25
iter: 60 | baseline loss: 1.9797 | candidate loss: 1.5755 | baseline train acc: 0.23 | candidate train acc: 0.35
iter: 80 | baseline loss: 2.1179 | candidate loss: 1.4597 | baseline train acc: 0.25 | candidate train acc: 0.32
iter: 100 | baseline loss: 2.1154 | candidate loss: 1.6904 | baseline train acc: 0.30 | candidate train acc: 0.27
iter: 120 | baseline loss: 1.8949 | candidate loss: 1.3554 | baseline train acc: 0.31 | candidate train acc: 0.42
iter: 140 | baseline loss: 1.6796 | candidate loss: 1.3079 | baseline train acc: 0.45 | candidate train acc: 0.48
iter: 160 | baseline loss: 1.96